In [70]:
import pandas as pd
import random

In [71]:
missed_entities_cv = pd.read_csv('../results/missed_entities_not_norm_entities_benchmark_cv_small.csv')
missed_entities_fleurs = pd.read_csv('../results/missed_entities_not_norm_entities_benchmark_fleurs_small.csv')
sentence_pool = pd.read_csv('../results/sentence_pool.csv')


In [ ]:
print(len(missed_entities_cv))
print(len(missed_entities_fleurs))


print(len(total_missed_entities))

860
1175
2035


In [73]:
total_missed_entities = total_missed_entities.drop_duplicates(subset='entity')
total_missed_entities.reset_index(drop=True, inplace=True)

In [74]:
total_missed_entities

,entity,candidate,ground_truth,type
0,Lebowski,Ni heter Lebauski och jag heter Lebauski.,"Så ni heter Lebowski, och jag heter Lebowski.",PER
1,Ljung,Djungeln dvärgbuske som gärna växer i sur jord,Ljung är en dvärgbuske som gärna växer i sur j...,PER
2,Mike,MIKE anslöt till Lidingö redan innan säsongen ...,Mike anslöt till Lidingö redan innan säsongen ...,PER
3,Östern,Vill du veta vad till exempel freden i östern ...,Vill du veta vad till exempel freden i Östern ...,LOC
4,Isaksson,Isakson väntade en tid men behövde sina pengar...,"Isaksson väntade en tid, men behövde sina peng...",PER
...,...,...,...,...
1393,Perry,Perli uttalade att han skulle återvända till T...,"""Perry uttalade att han skulle """"återvända til...",PER
1394,"Flagstaff, Arizona",Deras termiska beteende är inte lika stabilt s...,"""Deras termiska beteende är inte lika stabilt ...",LOC
1395,James Paget-sjukhuset,Han lades först in på sjukhus på James Pajet s...,Han lades först in på sjukhus på James Paget-s...,LOC
1396,Great Yarmouth,Han lades först in på sjukhus på James Pajet s...,Han lades först in på sjukhus på James Paget-s...,LOC


In [75]:
import pandas as pd
import json

import ast

def parse_metadata(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)  # Converts the string representation into a Python object
        except:
            return []
    return x

# Assuming sentence_pool is already a DataFrame with the structure you mentioned earlier
# Explode the metadata column for easier access and filtering
sentence_pool.reset_index(inplace=True)
sentence_pool['metadata'] = sentence_pool['metadata'].apply(parse_metadata)
print(type(sentence_pool.iloc[0]['metadata'])) 

sentence_pool['entity_type'] = sentence_pool['metadata'].apply(
    lambda items: items[0].get('entity_type') 
                  if isinstance(items, list) and items and isinstance(items[0], dict) 
                  else None
)

print(sentence_pool[['metadata', 'entity_type', 'sentence']].head())


buckets = {
    'PER': sentence_pool[sentence_pool['entity_type'] == 'PER'].to_dict('records'),
    'ORG': sentence_pool[sentence_pool['entity_type'] == 'ORG'].to_dict('records'),
    'LOC': sentence_pool[sentence_pool['entity_type'] == 'LOC'].to_dict('records'),
}

# Now buckets['PRS'], buckets['ORG'], etc. each hold a list of rows (as dicts) for that entity type.


# Now create buckets based on entity_type, filtering out rows where entity_type is None
# buckets = {
#     'PRS': sentence_pool_exploded[sentence_pool_exploded['entity_type'] == 'PRS'].to_dict('records'),
#     'ORG': sentence_pool_exploded[sentence_pool_exploded['entity_type'] == 'ORG'].to_dict('records'),
#     'LOC': sentence_pool_exploded[sentence_pool_exploded['entity_type'] == 'LOC'].to_dict('records'),
#     'OBJ': sentence_pool_exploded[sentence_pool_exploded['entity_type'] == 'OBJ'].to_dict('records')
# }

# Each key in 'buckets' will correspond to a list of dictionaries for each entity type


<class 'list'>
                                            metadata entity_type  \
0  [{'entity': 'Karlshamnsverket', 'entity_type':...         ORG   
1  [{'entity': 'Restaurang Terrassen Karlshamn', ...         LOC   
2     [{'entity': 'Jwan Ali', 'entity_type': 'PER'}]         PER   
3  [{'entity': 'Jacob Lagercrantz', 'entity_type'...         PER   
4  [{'entity': 'Fredriksson', 'entity_type': 'PER...         PER   

                                            sentence  
0  Karlshamnsverket producerade 2022 dubbelt så m...  
1  Inget gick att rädda efter den brand som start...  
2  Under lördagsförmiddagen kom ägaren Jwan Ali t...  
3  De gamla ska vara tömda och alla är dessutom k...  
4  Och Fredriksson pekar också på fördelen av när...  


In [76]:
for key, val in buckets.items():
    print(key, len(val))

PER 114581
ORG 45740
LOC 91445


In [82]:
def sample_sentences_by_entity_type(buckets, entity_type, n=10):
    samples_container = []
    sampled = random.sample(buckets[entity_type], n)
    sampled_indices = [row['index'] for row in sampled]  # keep track of original indices
    
    samples_container = [row['sentence'] for row in sampled]
    return samples_container, sampled_indices

samples, indicies = sample_sentences_by_entity_type(buckets, 'PER', n=10)
print(samples, indicies)


['Skulle det varit ett läkemedel som kom ut som skulle kunna halvera risken för demens skulle det vara jättestort  säger doktorand Hanna Wetterberg på Göteborgs universitet .', 'I handlingar från tingsrätten framkommer det att Soheil Naderi menar att hans bolag blev kapat .', 'I bussen fanns föraren  tre skolbarn och Madelen Dragsten som var på väg från dotterns skola hem till Djurås .', 'Då måste vi hjälpas åt  säger kommunalrådet Roger Fredriksson ( M ) .', 'Framgångsrik kirurg vid skönhetsklinik kritiseras av Ivo', 'Kommunens naturvårdsförvaltare Jochen Schreiber förstår varför sälgen kritiseras  men menar att träden är väldigt viktiga för naturens mångfald .', ' Man måste satsa på kärnuppdraget  att ta hand om våra patienter  säger Lars Edling  ordförande i Läkarföreningen i regionen .', ' Inga system är perfekta och det gäller även systemet för elstödet  säger Fredrik Falk  verksamhetschef på Försäkringskassan .', 'Även damerna kommer att få en ny tränare nästa säsong  det blir Ul

In [ ]:
"""
 PROMPT: I have one dataset of about 250k sentences with extracted entities referred to as sentence_pool, a row looks like this: | sentence: str | entities: list: str | metadata: list[{entity, entity_type}] |. I also have a dataframe called total_missed entities which consist of all entities we failed to recognize. The columns in that dataframe to consider is entity and type. Type is PER/ LOC/WRK etc. I want to create a new dataset according to the following instructions. For each entity in total_missed entities I want to sample 10 sentences containing the same entity_type. For example if the row in total_missed_entities is lebowski and the type is PER i want 10 candidate sentences containing a PER. I then want to send them as in a GPT call with a prompt asking it to pick the most appropriate sentence for that entity, and construct a new sentence with that entity, but with the entity replaced with our current entity. In this example the sentence chosen from the pool could be "Trots att mordvapen och vittnen saknas skulle åklagare Lena-Marie Bergström väcka åtal i dag " In which we want the new sentence to be "Trots att mordvapen och vittnen saknas skulle åklagare Lebowski väcka åtal i dag " The new newly constructed should contain three columns, sentence, entities and entities types"""

sentence    Karlshamnsverket producerade 2022 dubbelt så m...
entities                                 ["Karlshamnsverket"]
metadata    [{'entity': 'Karlshamnsverket', 'entity_type':...
Name: 0, dtype: object


In [ ]:

from dotenv import load_dotenv
from openai import OpenAI
import os
import re
import json 

load_dotenv()
# Assuming you have an OpenAI client instance initialized as:
api_key = os.getenv("TOMAS_OPENAI_KEY")
client = OpenAI(api_key=api_key)


PROMPT_TEMPLATE = (
        "You are given candidate sentences, each containing an entity of type '{entity_type}'.\n"
        "Your task is to select the sentence that best fits incorporating the missed entity '{missed_entity}'.\n"
        "Then, modify that sentence by replacing its original entity with '{missed_entity}', ensuring the sentence remains grammatically correct.\n"
        "\n"
        "ONLY RETURN the modified sentence.\n"
        "\n"
        "Candidate sentences:\n"
        "{candidate_sentences}\n"
    )

def create_sentence(sentences, entity_type, new_entity):
    prompt = PROMPT_TEMPLATE.format(
        entity_type=entity_type,
        missed_entity=new_entity,
        candidate_sentences="\n".join(f"{i}. {sentence}" for i, sentence in enumerate(sentences, 1))
    )
    
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    
    return response.choices[0].message.content.strip()



def create_new_sentences(sentence_pool: pd.DataFrame, total_missed_entities: pd.DataFrame) -> pd.DataFrame:
    
    new_rows = []
    
    # Process only the first 5 missed entities
    for _, missed_row in total_missed_entities.head(5).iterrows():
        missed_entity = missed_row['entity']
        entity_type = missed_row['type']
        
        #Get candidates
        for i in range(5):
            candidates, indicies = sample_sentences_by_entity_type(buckets, entity_type, n=10)
            sentence = create_sentence(candidates, entity_type, missed_entity)
        
            print(f"missed entity: {missed_entity} sentence {sentence}")
            
            new_rows.append({
                'sentence': sentence,
                'entity': missed_entity,
                'entity_type': entity_type
            })
            
    return pd.DataFrame(new_rows)

# Example usage:
new_dataset = create_new_sentences(sentence_pool, total_missed_entities)
#PROBLEM
# new_dataset.to_csv("new_dataset.csv", index=False)


missed entity: Lebowski sentence Är det någon som ska få slå mig är det Lebowski.
missed entity: Lebowski sentence 'När Firenze hörde av sig så var det nästan ett solklart val, säger Lebowski till SVT Sport.'
missed entity: Lebowski sentence Nu står det klart att det är Ronnebybördiga Lebowski som blir ny generaldirektör för Kustbevakningen, det meddelades av regeringen under torsdagen.
missed entity: Lebowski sentence 'Du dök inte ens upp på begravningen för Lebowski som beredde vägen för dig att få det liv som du har skriver en person på Kakas Instagram något som dragit till sig 5000 kommentarer .'
missed entity: Lebowski sentence Lebowski utmanar Donald Trump, vill bli republikansk presidentkandidat.
missed entity: Ljung sentence Ljung lämnade Malmö FF efter den gångna säsongen  24-åringen som har kontrakt med ryska Rostov har länge ryktats vara på väg till Spanien  och under onsdagen blev det officiellt .
missed entity: Ljung sentence I början av januari lade Ljung fram reformen, s

In [87]:
new_dataset

,sentence,entity,entity_type
0,Är det någon som ska få slå mig är det Lebowski.,Lebowski,PER
1,'När Firenze hörde av sig så var det nästan et...,Lebowski,PER
2,Nu står det klart att det är Ronnebybördiga Le...,Lebowski,PER
3,'Du dök inte ens upp på begravningen för Lebow...,Lebowski,PER
4,"Lebowski utmanar Donald Trump, vill bli republ...",Lebowski,PER
5,Ljung lämnade Malmö FF efter den gångna säsong...,Ljung,PER
6,"I början av januari lade Ljung fram reformen, ...",Ljung,PER
7,'Ljung är ansvarig i Jönköping och Huskvarna.',Ljung,PER
8,Ljung den brittiska designern som anses ha ska...,Ljung,PER
9,"En dålig idé, säger oppositionsrådet Ljung (S).",Ljung,PER
